# [Molecular Simulation with OpenMM](https://github.com/bacalhau-project/examples/tree/main/molecular-dynamics)

* [Open In Colab](https://colab.research.google.com/github/bacalhau-project/examples/blob/main/molecular-dynamics/openmm/index.ipynb)
* [Open In Binder](https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=molecular-dynamics/openmm/index.ipynb)

[OpenMM](https://github.com/openmm/openmm) is a toolkit for molecular simulation. It is a physic based libraries that is useful for refining the structure and exploring functional interactions with other molecules. It provides a combination of extreme flexibility (through custom forces and integrators), openness, and high performance (especially on recent GPUs) that make it truly unique among simulation codes.

## Protein data

We use a processed 2DRI dataset that represents the ribose binding protein in bacterial transport and chemotaxis. The source organism is the [Escherichia coli](https://en.wikipedia.org/wiki/Escherichia_coli) bacteria.
You can find more details on this protein at the related [RCSB Protein Data Bank page](https://www.rcsb.org/structure/2dri).

![image.png](./2dri-image.png)

Protein data can be stored in a `.pdb` file, this is a human readable format. It provides for description and annotation of protein and nucleic acid structures including atomic coordinates, secondary structure assignments, as well as atomic connectivity.
See more information about PDB format [here](https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html).

We are printing the first 10 lines of the file. The output contains a number of ATOM records. These describe the coordinates of the atoms that are part of the protein.

In [ ]:
%%bash
head ./2dri-processed.pdb

REMARK   1 CREATED WITH OPENMM 7.6, 2022-07-12
CRYST1   81.309   81.309   81.309  90.00  90.00  90.00 P 1           1 
ATOM      1  N   LYS A   1      64.731   9.461  59.430  1.00  0.00           N  
ATOM      2  CA  LYS A   1      63.588  10.286  58.927  1.00  0.00           C  
ATOM      3  HA  LYS A   1      62.707   9.486  59.038  1.00  0.00           H  
ATOM      4  C   LYS A   1      63.790  10.671  57.468  1.00  0.00           C  
ATOM      5  O   LYS A   1      64.887  11.089  57.078  1.00  0.00           O  
ATOM      6  CB  LYS A   1      63.458  11.567  59.749  1.00  0.00           C  
ATOM      7  HB2 LYS A   1      63.333  12.366  58.879  1.00  0.00           H  
ATOM      8  HB3 LYS A   1      64.435  11.867  60.372  1.00  0.00           H  


## Write the script
To run the script above all we need is a Python environment with the [OpenMM library](http://docs.openmm.org/latest/userguide/application/01_getting_started.html) installed.

In [ ]:
%%bash
conda install -c conda-forge openmm

In [ ]:
%%bash
python -m openmm.testInstallation


OpenMM Version: 8.0
Git Revision: a7800059645f4471f4b91c21e742fe5aa4513cda

There are 3 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 OpenCL - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.30471e-06
Reference vs. OpenCL: 6.74827e-06
CPU vs. OpenCL: 7.07625e-07

All differences are within tolerance.


In [ ]:
import os
from openmm import *
from openmm.app import *
from openmm.unit import *

# Input Files
input_path = './2dri-processed.pdb'
os.path.exists(input_path) # check if input file exists
pdb = PDBFile(input_path)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# Output
output_path = './final_state.pdbx'
if not os.path.exists(os.path.dirname(output_path)): # check if ouput dir exists
    os.makedirs(os.path.dirname(output_path))

# System Configuration

nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 1.5*amu

# Integration Options

dt = 0.002*picoseconds
temperature = 310*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

# Simulation Options

steps = 10
equilibrationSteps = 0
#platform = Platform.getPlatformByName('CUDA')
platform = Platform.getPlatformByName('CPU')
#platformProperties = {'Precision': 'single'}
platformProperties = {}
dcdReporter = DCDReporter('trajectory.dcd', 1000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, elapsedTime=True, remainingTime=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 1000)

# Prepare the Simulation

print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform, platformProperties)
simulation.context.setPositions(positions)

# Minimize and Equilibrate

print('Performing energy minimization...')
simulation.minimizeEnergy()
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

# Simulate

print('Simulating...')
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.step(steps)

# Write file with final simulation state

state = simulation.context.getState(getPositions=True, enforcePeriodicBox=system.usesPeriodicBoundaryConditions())
with open(output_path, mode="w+") as file:
    PDBxFile.writeFile(simulation.topology, state.getPositions(), file)
print('Simulation complete, file written to disk at: {}'.format(output_path))

Building system...
Performing energy minimization...
Equilibrating...
Simulating...
Simulation complete, file written to disk at: ./final_state.pdbx


In [ ]:
%%bash
head ./final_state.pdbx

data_cell
# Created with OpenMM 8.0, 2023-03-12
#
_cell.length_a        81.3090
_cell.length_b        81.3090
_cell.length_c        81.3090
_cell.angle_alpha     90.0000
_cell.angle_beta      90.0000
_cell.angle_gamma     90.0000
#


---

In [ ]:
%%bash
pip install markdown

In [ ]:
import markdown

In [ ]:
markdown.markdown(
    '[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bacalhau-project/examples/blob/main/molecular-dynamics/openmm/index.ipynb)'
)

'<p><a href="https://colab.research.google.com/github/bacalhau-project/examples/blob/main/molecular-dynamics/openmm/index.ipynb"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg" /></a></p>'

In [ ]:
markdown.markdown(
    '[![Open In Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=molecular-dynamics/openmm/index.ipynb)'
)

'<p><a href="https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=molecular-dynamics/openmm/index.ipynb"><img alt="Open In Binder" src="https://mybinder.org/badge.svg" /></a></p>'